In [1]:
!pip install -q -r requirements.txt

In [ ]:
import json
import unicodedata
import pandas as pd

In [ ]:
data = json.load(open('data/ner-wikipedia-dataset/ner.json', 'r'))

type_map = {
    'その他': 0.0,
    '人名': 1.0,
    '法人名': 2.0,
    '政治的組織名': 3.0,
    'その他の組織名': 4.0,
    '地名': 5.0,
    '施設名': 6.0,
    '製品名': 7.0,
    'イベント名': 8.0
    }

In [ ]:
data[0:2]

[{'curid': '3572156',
  'entities': [{'name': 'SPRiNGS', 'span': [0, 7], 'type': 'その他の組織名'}],
  'text': 'SPRiNGSと最も仲の良いライバルグループ。'},
 {'curid': '2415078',
  'entities': [{'name': 'レッドフォックス株式会社', 'span': [0, 12], 'type': '法人名'},
   {'name': '東京都千代田区', 'span': [14, 21], 'type': '地名'}],
  'text': 'レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。'}]

In [ ]:
def ner_preprocess(record):
    '''
    固有表現(inside)とその他(outside)に分けて、
    それらに対応付く変数を作っていく
    '''
    curid = record['curid']
    entities = record['entities']
    text_body = record['text']
    texts, labels, start_, end_ = [], [], [], []
    pos = 0
    for entity in entities:
        start = entity['span'][0]
        end = entity['span'][1]
        label = entity['type']
        texts.append(text_body[pos:start]); labels.append('その他')
        start_.append(pos); end_.append(start)
        texts.append(text_body[start:end]); labels.append(label)
        start_.append(start); end_.append(end)
        pos = end
    texts.append(text_body[pos:len(text_body)]); labels.append('その他')
    start_.append(pos); end_.append(len(text_body))
    curids = [curid] * len(texts)
    text_bodies = [text_body] * len(texts)
    return curids, text_bodies, texts, labels, start_, end_

def text_normalize(text):
    text = text.strip()
    text = unicodedata.normalize('NFKC', text)
    return text

def text_preprocess(df, type_map):
    df['text_body'] = df['text_body'].map(text_normalize)
    df['text'] = df['text'].map(text_normalize)
    df['label'] = df['label'].map(type_map)
    return df

In [ ]:
def json_to_df(data, type_map):
    curid_arr, text_body_arr, text_arr, label_arr, start_arr, end_arr = [], [], [], [], [], []
    for record in data:
        curids, text_bodies, texts, labels, start_, end_ = ner_preprocess(record)
        curid_arr.extend(curids)
        text_body_arr.extend(text_bodies)
        text_arr.extend(texts)
        label_arr.extend(labels)
        start_arr.extend(start_)
        end_arr.extend(end_)
    data_df = pd.DataFrame(
        {
         'curid': curid_arr, 
         'text_body': text_body_arr, 
         'text': text_arr, 
         'start': start_arr, 
         'end': end_arr, 
         'label': label_arr
         }
        )
    data_df = text_preprocess(data_df, type_map)
    data_df.to_csv('data/preprocessed_data.csv', index=False)

In [ ]:
json_to_df(data, type_map)
data_df = pd.read_csv('data/preprocessed_data.csv').dropna(how='any')
data_df.reset_index(drop=True, inplace=True)
data_df

,curid,text_body,text,start,end,label
0,3572156,SPRiNGSと最も仲の良いライバルグループ。,SPRiNGS,0,7,4.0
1,3572156,SPRiNGSと最も仲の良いライバルグループ。,と最も仲の良いライバルグループ。,7,23,0.0
2,2415078,レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。,レッドフォックス株式会社,0,12,2.0
3,2415078,レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。,は、,12,14,0.0
4,2415078,レッドフォックス株式会社は、東京都千代田区に本社を置くITサービス企業である。,東京都千代田区,14,21,5.0
...,...,...,...,...,...,...
29836,92497,地域WiMAXサービスは延岡市の主要地、日本のKDDI「ケーブルプラスWiMAX」エリア内で...,」エリア内で利用可能。,40,51,0.0
29837,243429,ダニス・タノヴィッチはユーゴスラビア出身の映画監督・脚本家である。,ダニス・タノヴィッチ,0,10,1.0
29838,243429,ダニス・タノヴィッチはユーゴスラビア出身の映画監督・脚本家である。,は,10,11,0.0
29839,243429,ダニス・タノヴィッチはユーゴスラビア出身の映画監督・脚本家である。,ユーゴスラビア,11,18,5.0


In [ ]:
data_df.to_csv('data/preprocessed_data.csv', index=False)